In [5]:
package 'Structure'{
    import ScalarValues::* ; 
    
    value type Data;
    value type 'Ground Station Ping';
    
    package 'Interfaces'{ 
        port def CommunicationInterface{
            value out data:Data; 
            value in ping:'Ground Station Ping';
        }
        
        interface def CommunicationLink { //defining CommunicationLink INTERFACE which is an association block whose ends are ports
            end groundPort : CommunicationInterface;
            end spacePort : CommunicationInterface;
            stream spacePort::data to groundPort::data;//not sure about these. should I define value type ping
            stream groundPort::ping to spacePort::ping;  //    in both blocks 
        }
    }
    
    
    
    block 'Mission'{ //Mission is made of 2 parts, communicating via CommunicationLink
        
        part groundStation : 'Ground Station' {
            port groundCommIF redefines CommIF;
            exhibit 'Ground Station'::groundStationModes(
                in groundStati0n = self);
        }
        
        part spacecraftVehicle : 'Spacecraft Vehicle'{
            port spaceCommIF redefines CommIF1;
            exhibit 'Spacecraft Vehicle'::spacecraftModes(
                in SCvehicle = self);
        }
        
        interface : Interfaces::CommunicationLink connect            //creating interface
            groundPort => groundStation::groundCommIF to 
            spacePort  => spacecraftVehicle::spaceCommIF; 
    }
    

    
    
    block 'Ground Station'{
        port CommIF : ~Interfaces::CommunicationInterface;      
        
        value type 'Start Signal';
        
        activity PingSpacecraft(){
            // 1. send signal on CommunicationLink interface 
           send 'Ground Station Ping'() to CommIF;
        } 
        
        
        activity ReceiveData(){
            // 0. merge node
            // 1. waiting for a trigger : Data Packet arriving on CommunicationLink
            // 2. process data (flow dataframe from port to processing action)
                // (flow trigger::dataPacket to process::dataPacket;)
            // 3. repeat (recursive ? or flow to merge node)
            first start;
            then merge repeat;
            
            then accept DataPacket (dataFrame : Data);
            
            then action processData(argument flow from DataPacket::dataFrame);
            
            then repeat;
        }
        
        
        state groundStationModes(in groundStation:'Ground Station'){
            entry action initial; then idle;
            
            state idle;
            
            state operation{
                entry action : groundStation::PingSpacecraft;
                do action : groundStation::ReceiveData;
            }
            
            transition 'idle-operation'
                first idle
                accept 'after(30s)' //not implemented yet, will fail
                then operation;
            transition 'idle-operation_2'
                first idle
                accept 'Start Signal'
                then operation;
            
            transition 'operation-idle'
                first operation
                accept 'Spacecraft Vehicle'::'Shut Down Signal'
                then idle;
        }// end state groundStationModes
    }
    
    
    
    
    block 'Spacecraft Vehicle'{
        port CommIF1 : Interfaces::CommunicationInterface;
       
        value type 'Battery Fully Charged';
        value type 'Shut Down Signal';
        value type 'Battery Low';
        value type 'Transmission Done Signal';
               
        
        
        activity ConsumePower(){
            // 0. in case trigger:DeviceOff => "stop action"
            // 1. merge node
            // 2. battery--;
            // 3a. case battery <40 : send signal 'Battery Low' to TransmitData process
            // 3b. case battery >40 : flow to merge node 1.
            
            //this will run in the background (I hope)
            accept shutDown(shutDownSignal : 'Shut Down Signal');
            then done; 
            
            merge repeat;
            then action dechargeBattery{/* battery--; */}
            then decide;
                if battery>= 40 then repeat;
                else sendBatteryLow;
                    
            send sendBatteryLow of 'Battery Low'() to Structure::Mission::spacecraftVehicle;
        } //seems weird for me to send it there but I wouldn't know where else to send it

        
        
        
        activity RechargeBatteries(){
            // 1. merge node
            // 2a. case battery >=100 : send signal 'Battery fully charged' then stop action
            // 2b. case battery <100 : battery++; then flow to merge node 1
            first start;
            then merge mergeNode;
            
            then decide;
                if battery>= 100 then sendCharged;
                else chargeBattery;
            
            action chargeBattery{ /* battery++; */ }
            then mergeNode;
            
            send sendCharged of 'Battery Fully Charged'() to Structure::Mission::spacecraftVehicle;
            then done;
        }
        
        activity SendDataFrames(){
            // 1. data = data-1024;
            // 2. send DataPacket on CommunicationInterface to Ground Station 
            
            first start;
            
            fork parallelStart;
            
            succession parallelStart then datasub;
            succession parallelStart then sendDataPacket;
            succession datasub then parallelEnd;
            succession sendDataPacket then parallelEnd;
            
            fork parallelEnd;
            then done;
            
            
            action datasub(){/* data = data-1024; */ }
            action sendDataPacket(){ send Data() to CommIF1; }
            
        }
        
        
        
        activity TransmitData(){
            // 00 trigger "battery low" : go to 6.Merge
            // 1. run small Javascript Expression
            // 2. Fork (consume power + the following)
            // 3. Merge
            // 4. Decision
            // 5. Case data>0 : sendDataFrames() then go back to 3.Merge
            // 5. Case data=0 : Javascript expression
            // 6. Merge 
            // 7. Send signal Device Off to activity ConsumePower
            // 8. Send signal Transmission Done  
            
            first start;
            then action js_1(){ /* javascript expression set t_initial */ }
            
            then fork fork1;
            succession fork1 then consumePower;
            succession fork1 then repeat;
                
            merge repeat;
            then decide;
                if data>0 then sendDataFrames;
                else js_2;
                
            action js_2(){ /* javascript expression : t_final transmission done  */ }
            then merge mergeNode_2;
            then send 'Shut Down Signal'() to Structure::Mission::spacecraftVehicle;
            then send 'Transmission Done Signal'() to Structure::Mission::spacecraftVehicle;
            
            action consumePower:ConsumePower;
            action sendDataFrames:SendDataFrames;
            then repeat;
            
            accept batteryLow(batteryLowSignal:'Battery Low');
            then mergeNode_2;
        }
        
        state spacecraftModes(
            in SCvehicle:'Spacecraft Vehicle'){
            
            state dataTransit {
                entry action initial; then waitingGSPing;
                
                state waitingGSPing;
                transition 'waitingGSPing-transmitting'
                    first waitingGSPing
                    accept 'Ground Station Ping'
                    // if inState not recharging? // no notation yet
                    then transmitting;
                
                state transmitting{
                    do action : SCvehicle::TransmitData;
                }
                
                transition 'transmitting-waitingGSPing'
                    first transmitting
                    accept 'Transmission Done Signal'
                    then waitingGSPing;
                transition 'transmitting-waitingGSPing_2'
                    first transmitting
                    accept 'Battery Low'
                    then waitingGSPing;
            } // end state spacecraftModes
            
            state charging{
                entry action initial; then notRecharging;
                
                state notRecharging;
                transition 'notRecharging-charging'
                    first notRecharging
                    accept 'when(battery < 80)' //not implemented yet, will fail
                    then recharging;
                state recharging{
                    do action : SCvehicle::RechargeBatteries;
                }
                transition 'charging-notRecharging'
                    first charging
                    accept 'Battery Fully Charged'
                    then notRecharging;
            }
        }
                
        value data : Real = 102400;
        value battery : Real = 100;
        value transmissionInProgress : Boolean=false;
        value t0 : Real = 0;
        value tfinal : Real = 0;
    }
}

ERROR:Couldn't resolve reference to Type ''after(30s)''. (5.sysml line : 85 column : 24)
ERROR:Couldn't resolve reference to Type ''when(battery < 80)''. (5.sysml line : 247 column : 28)
